# stacking堆叠集成DNN

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
x_temp1 = pd.read_csv('//content//drive//MyDrive//data//train_final.csv', engine='python')
x_temp2 = pd.read_csv('//content//drive//MyDrive//data//test_final.csv', engine='python')

In [ ]:
train_DL = x_temp1.copy()
test_DL = x_temp2.copy()

## 

In [ ]:
train_SDNN = x_temp1.copy()
test_SDNN = x_temp2.copy()
train_SDNN.fillna(0,inplace=True)
test_SDNN.fillna(0,inplace=True)

X_train = train_SDNN.drop(columns=['loan_status']).values
Y_train = train_SDNN['loan_status'].values.astype(int)
X_test = test_SDNN.drop(columns=['loan_status']).values
Y_test = test_SDNN['loan_status'].values.astype(int)

from tensorflow.keras.utils import to_categorical
#Y_test=to_categorical(Y_test,2).astype(int)
#Y_train=to_categorical(Y_train,2).astype(int)


In [ ]:
import keras as K
from keras.layers.core import Dropout
init = K.initializers.glorot_uniform(seed=1)
model = K.models.Sequential()
model.add(K.layers.Dense(units=146, input_dim=2, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=147, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='softmax'))
model.compile(loss='categorical_crossentropy',  metrics=['accuracy'])


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gdbt_clf = GradientBoostingClassifier(random_state = 20)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
nby_clf = GaussianNB()

In [ ]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier()

In [ ]:
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold):
  folds=StratifiedKFold(n_splits=n_fold,random_state=1)
  test_pred=np.empty((test.shape[0],1),float)
  train_pred=np.empty((0,1),float)
  for train_indices,val_indices in folds.split(train,y.values):
    
    x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
    y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

    model.fit(X=x_train,y=y_train)
    train_pred=np.append(train_pred,model.predict(x_val))
    test_pred=np.column_stack((test_pred,model.predict(test)))
  test_pred_a=np.mean(test_pred,axis=1) #按行计算均值
  return test_pred_a.reshape(-1,1),train_pred


In [ ]:
x_train_2=train_SDNN.drop(columns=['loan_status'])
x_test_2=test_SDNN.drop(columns=['loan_status'])
y_train_2=train_SDNN['loan_status']

test_pred3 ,train_pred3 =Stacking(model=ada_clf,n_fold=10,train=x_train,test=x_test,y=y_train)
print(test_pred3.size)
train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)

test_pred4 ,train_pred4 =Stacking(model=nby_clf,n_fold=10,train=x_train,test=x_test,y=y_train)
print(test_pred4.size)
train_pred4=pd.DataFrame(train_pred4)
test_pred4=pd.DataFrame(test_pred4)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


50000


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


50000


In [ ]:
dff_2 = pd.concat([train_pred3, train_pred4], axis=1)
dff_test_2 = pd.concat([test_pred3, test_pred4], axis=1)

In [ ]:
model.fit(dff_2,y_train_2)

1563/1563 [==============================] - 2s 1ms/step - loss: 0.0000e+00 - accuracy: 0.7958


In [ ]:
eval_2 = model.evaluate(dff_test_2, Y_test, verbose=1)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" \
          % (eval_2[0], eval_2[1] * 100) )

1563/1563 [==============================] - 2s 1ms/step - loss: 0.0000e+00 - accuracy: 0.8045
Evaluation on test data: loss = 0.000000 accuracy = 80.45% 

